# API Data Extraction

Using all the scraped movie ID with the OMDB API to get movie information for each title.

In [1]:
from urllib2 import Request, urlopen, URLError
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

**Import the previously scraped movie id along with the title**

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data.head()

,id,name
0,tt0110912,Pulp Fiction
1,tt1872181,The Amazing Spider-Man 2
2,tt0111161,The Shawshank Redemption
3,tt0076759,Star Wars: Episode IV - A New Hope
4,tt0088763,Back to the Future


In [4]:
data['id'][0]

'tt0110912'

**Generate a list of api calls that will be sent out**

In [5]:
api_calls = []

for ids in range(len(data)):
    api_calls.append('http://www.omdbapi.com/?i='+data['id'][ids]+'&plot=full')

In [6]:
api_calls[:5]

['http://www.omdbapi.com/?i=tt0110912&plot=full',
 'http://www.omdbapi.com/?i=tt1872181&plot=full',
 'http://www.omdbapi.com/?i=tt0111161&plot=full',
 'http://www.omdbapi.com/?i=tt0076759&plot=full',
 'http://www.omdbapi.com/?i=tt0088763&plot=full']

**Create a function to go through the API call and return a singular dataframe object**

In [7]:
def get_row(api_call):
    request = Request(api_call)
    response = urlopen(request)
    info = response.read()
    row = pd.read_json(info, typ='series', orient='values')
    return row

** Have the loop go through all the api calls that were previously generated and append them to a larger dataframe**

Keep track of failed API calls, in the end only 3 failed.

In [18]:
movies = pd.DataFrame()

In [19]:
counter = 1
for ac in api_calls:
    try:
        temp = get_row(ac)
        movies = pd.concat([movies,temp], axis=1, ignore_index=True )
        counter +=1
        if(counter%500 == 0):
            print counter, 'DONE'
    except:
        print 'skipped:',counter
        counter+=1
        continue

500 DONE
1000 DONE
1500 DONE
2000 DONE
2500 DONE
3000 DONE
3500 DONE
4000 DONE
4500 DONE
5000 DONE
skipped: 5344
5500 DONE
6000 DONE
6500 DONE
7000 DONE
7500 DONE
8000 DONE
8500 DONE
skipped: 8587
skipped: 8618
9000 DONE
9500 DONE


In [20]:
movies = movies.transpose()

**Saved all extracted info and move on to the cleaning stage**

In [38]:
file_name = '../project-6-apis-randomforests/movieinfo.csv'
movies.to_csv(file_name, index = False, encoding='utf-8')